In [13]:
%config IPCompleter.greedy = True
%autosave 60

Autosaving every 60 seconds


## 1. NLP

### Cargar archivo

In [54]:
import pandas as pd
import os
ruta_archivo = os.path.join("imdb_dataset.csv")
df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2').sample(2000, replace=False)
#df_criticas = pd.read_csv(ruta_archivo, encoding='iso-8859-2')
df_criticas

,Review,Label
9843,"An awful film; badly written, badly acted, cli...",neg
29611,I have a 5 minute rule (sometimes I'll leave l...,neg
25161,Steven Seagal appears to be sleepwalking throu...,neg
17579,A classic late 50's film. The superannuated he...,pos
36745,I rented this movie roughly 4-5 years ago and ...,neg
...,...,...
19446,I originally saw this film while I was working...,pos
22262,About as hilarious as 50s British comedy can g...,pos
23288,Although the premise of the movie involves a m...,pos
44201,"Well, it definitely is unlike anything else di...",pos


### Palabras de parada

In [55]:
import nltk
import string
# nltk.download('punkt')
# nltk.download('stopwords')
palabras_de_parada_ingles = set(nltk.corpus.stopwords.words('english') + list(string.punctuation) + ['...', '..'])
palabras_de_parada_ingles

{'!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 '+',
 ',',
 '-',
 '.',
 '..',
 '...',
 '/',
 ':',
 ';',
 '<',
 '=',
 '>',
 '?',
 '@',
 '[',
 '\\',
 ']',
 '^',
 '_',
 '`',
 'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 '

### Normalizar el texto

In [57]:
import re

def descontraer(frase):
    # contracciones específicas
    frase = re.sub(r"won\'t", "will not", frase)
    frase = re.sub(r"can\'t", "cannot", frase)
    frase = re.sub(r"y\'all", "you all", frase)

    # contracciones generales
    frase = re.sub(r"n\'t", " not", frase)
    frase = re.sub(r"\'re", " are", frase)
    # las contracciones 's también podrían ser has
    frase = re.sub(r"\'s", " is", frase)
    # las contracciones 'd también podrían ser had
    frase = re.sub(r"\'d", " would", frase)
    frase = re.sub(r"\'ll", " will", frase)
    frase = re.sub(r"\'t", " not", frase)
    frase = re.sub(r"\'ve", " have", frase)
    frase = re.sub(r"\'m", " am", frase)
    return frase

In [58]:
def limpiar_html(raw_html):
    reg_limpiar = re.compile('<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});')
    texto_sin_html = re.sub(reg_limpiar, '', raw_html)
    return texto_sin_html

def limpiar_texto(critica):
    texto_descontraido = descontraer(critica)
    texto_limpio = limpiar_html(texto_descontraido)
    return texto_limpio

def normalizar_critica(critica):
    critica_texto_limpio = limpiar_texto(critica)
    tokens = nltk.tokenize.casual.casual_tokenize(critica_texto_limpio, "english")
    tokens_normalizados = [token.lower() for token in tokens if token not in palabras_de_parada_ingles]
    return " ".join(tokens_normalizados)

def normalizar_fila(fila):
    nueva_fila = fila
    critica_normalizada = normalizar_critica(fila['Review'])
    nueva_fila['Review'] = critica_normalizada
    return nueva_fila

In [59]:
df_criticas = df_criticas.apply(normalizar_fila, axis=1)
df_criticas

,Review,Label
9843,an awful film badly written badly acted cliche...,neg
29611,i 5 minute rule sometimes i leave leway 10 if ...,neg
25161,steven seagal appears sleepwalking dreadful mo...,neg
17579,a classic late 50 film the superannuated headl...,pos
36745,i rented movie roughly 4-5 years ago instantly...,neg
...,...,...
19446,i originally saw film i working musician music...,pos
22262,about hilarious 50s british comedy get the bel...,pos
23288,although premise movie involves major coincide...,pos
44201,well definitely unlike anything else directed ...,pos


### Obtener el vocabulario del problema

In [60]:
def obtener_vocabulario_problema(df_criticas_normalizadas):
    cuerpo_texto_normalizado = ' '.join(df_criticas_normalizadas['Review'].tolist())
    vocabulario_problema = cuerpo_texto_normalizado.split()
    vocabulario_ordenado = sorted(set(vocabulario_problema))
    return vocabulario_ordenado

def obtener_vocabulario_problema_y_posicion(vocabulario_del_problema_ordenado):
    vocabulario_y_posicion = {}
    for i, token in enumerate(vocabulario_del_problema_ordenado):
        vocabulario_y_posicion[token] = i
    return vocabulario_y_posicion

In [61]:
vocabulario_problema = obtener_vocabulario_problema_y_posicion(obtener_vocabulario_problema(df_criticas))
len(vocabulario_problema)

30017

### One Hot Encoding

In [62]:
import numpy as np

def obtener_one_hot_vector(critica, vocabulario_problema_y_posicion):
    one_hot_vector = np.zeros(len(vocabulario_problema_y_posicion), dtype=int)
    for token in critica.split():
        one_hot_vector[vocabulario_problema_y_posicion[token]] = 1
    return one_hot_vector

In [63]:
one_hots = []
indices = []
for indice, fila in df_criticas.iterrows():
    one_hot = obtener_one_hot_vector(fila['Review'], vocabulario_problema)
    indices.append(indice)
    one_hots.append(one_hot)

## 2. Crear y entrenar el modelo predictivo

In [64]:
# se cambia el nombre de la variable 'one_hots' para mas claridad
X = one_hots
Y = df_criticas['Label'].ravel()

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [66]:
import gc
# Liberar memoria RAM
lst = [df_criticas]
del df_criticas, one_hots, X, Y
del lst
gc.collect()

198

In [67]:
from sklearn.linear_model import LogisticRegression
clasificador_reg_log = LogisticRegression(random_state=0, solver='liblinear') #Falta añadir el solver del magister...
clasificador_reg_log.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [74]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
predicciones = clasificador_reg_log.predict(X_test)
metricas = {
    'accuracy': accuracy_score(Y_test, predicciones),
    'ṕrecision': precision_score(Y_test, predicciones, pos_label='pos'),
    'recall': recall_score(Y_test, predicciones, pos_label='pos'),
    'f1': f1_score(Y_test, predicciones, pos_label='pos')
}
print(metricas)

{'accuracy': 0.8366666666666667, 'ṕrecision': 0.8286604361370716, 'recall': 0.86084142394822, 'f1': 0.8444444444444443}


### Guardar el clasificador como un archivo

In [ ]:
import pickle
ruta_archivo_clasificador = os.path.join('clasificador-regresion-logistica.pkl')
archivo_clasificador = open(ruta_archivo_clasificador, 'wb')
pickle.dump(clasificador_reg_log, archivo_clasificador)
archivo_clasificador.close()

### Guardar el vocabulario del problema como un archivo

In [ ]:
ruta_archivo_vocabulario = os.path.join('vocabulario-problema.pkl')
archivo_vacabulario = open(ruta_archivo_vocabulario, 'wb')
pickle.dump(vocabulario_problema, archivo_vacabulario)
archivo_vacabulario.close()

### Guardar las palabras de parada como un archivo

In [ ]:
ruta_archivo_palabras_parada = os.path.join('palabras-parada.pkl')
archivo_palabras_parada = open(ruta_archivo_palabras_parada, 'wb')
pickle.dump(palabras_de_parada_ingles, archivo_palabras_parada)
archivo_palabras_parada.close()